Purpose of this notebook: 
* compute the offsets from the different summary stat methods

In [169]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [5]:
%autoreload 2
%matplotlib inline 
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
plt.rcParams['figure.figsize'] = (10, 8)

In [6]:
from __future__ import print_function
import pandas as pd
import h5py
import sys
sys.path.append("../")
import extract_catalog as ec
import get_DM_centroids as getDM
# import seaborn as sns
import compute_distance as compDist
import plot_cred_int as plotCI
import get_KDE

In [7]:
data_path = "../../data/test_results/"

# read in files 

want to read in the second half of the projections for the DM data

In [10]:
clstNo = 43
date_stamp = '06_21_16'

DM_fhats = \
    h5py.File(data_path +
              "DM_fhat_clst{0}_{1}.h5".format(
            clstNo, date_stamp), 'r')
star_fhats = \
    h5py.File(data_path +
              "stars_fhat_clst{0}_{1}.h5".format(
            clstNo, date_stamp), 'r')
main_FOF_h5 = h5py.File(
        "../../data/" +
        "Illustris-1_fof_subhalo_myCompleteHaloCatalog_00135" +
        ".hdf5", "r")

# figure out how many projections we did 

In [11]:
clstNo = [int(no) for no in star_fhats.keys()]

In [12]:
star_paths = compDist.retrieve_cluster_path(star_fhats)

the list of `DM_paths` are 

The projections do not mean a thing except for debugging purposes, 
only the `clstNo` is a valid identifier 

In [14]:
split_path = star_paths[0].split('/')

In [15]:
fixed_path = '/'.join(split_path[1:4])

In [16]:
path = '0' + '/' + fixed_path

In [17]:
projections = star_fhats[str(clstNo[0]) + '/' + fixed_path].keys()

## columns in my uber dataframe (projection independent part)
| ClstNo | $m_{200c}$ | richness | relaxedness0| relaxedness1 |  


## projection dependent part 
| ClstNo | kernel_width | $\Delta s_{BCG}$ | $\Delta s_{KDE}$ | $\zeta$ | $\Delta s_{SA}$ | $\Delta s_{C}$ |  projection | total_peaks_dens | 

ClstNo is retained to match and join the two sets of `df`

# gather the projection dependent part of the `uber_df`

In [18]:
summary_stat_keys = ['BCG', 'centroid', "shrink_cent"]

In [19]:
df_list = []
for star_path in star_paths: 
    clstNo = [int(star_path.split('/')[0])]
    star_fhat = star_fhats[star_path]
    for bin_width in ['/0.0/', '/25.0/']:
        DM_fhat = DM_fhats[star_path + bin_width]
    
        # temporarily put results in matched_stat first
        matched_stat = \
            compDist.compute_distance_between_DM_and_gal_peaks(
                star_fhat, DM_fhat, compute_2D_distance=True
            )        
            
        # compute all other offsets before putting them somewhere 
        # for computing my uber_df
        dist_dict = compDist.compute_distance_for_other_peaks(
            matched_stat, star_fhat, summary_stat_keys=summary_stat_keys,
            compute_2D_distance=True
        )
        peak_no = len(matched_stat['dist'])
        if peak_no > 1:
            df = pd.DataFrame(dist_dict, 
                              index=[clstNo[0] for i in range(peak_no)])
            df['peak_id'] = range(peak_no)
            df['KDE' ] = matched_stat['dist']
            df['Delta_x_KDE' ] = matched_stat['Delta_x_KDE']
            df['Delta_y_KDE' ] = matched_stat['Delta_y_KDE']
            
        else:
            df = pd.DataFrame(dist_dict, index=clstNo)
            df['peak_id'] = 0.
            df['KDE'] = matched_stat['dist'][:1]
            df['Delta_x_KDE' ] = matched_stat['Delta_x_KDE'][:1]
            df['Delta_y_KDE' ] = matched_stat['Delta_y_KDE'][:1]
            
        df['total_peaks_dens'] = np.sum(star_fhat['peaks_dens'])
        df['bin_width'] = float(bin_width[1:-1])    
        df_list.append(df)

uber_df_proj = pd.concat(df_list)
uber_df_proj.to_hdf('../../data/uber_df_{0}.h5'.format(date_stamp), 
                    'df')
